# 🚀 Quick Start - Panel Dashboard

**最新版本**: Phase 2.0 - Panel Integration  
**特性**: 可导出静态 HTML（控件 + 图表都可交互）

---

## 📋 本 Notebook 的步骤

1. **初始化环境** - 加载框架 + CSS 宽度修复
2. **加载数据** - 从 Parquet 文件加载
3. **生成基本 AI Context** - 数据结构信息
4. **生成完整 AI Prompt** - 包含所有文档（复制给 AI）
5. **选择维度** - 选择分析维度
6. **创建 Panel 仪表盘** - AI 生成分析逻辑
7. **导出 HTML** - 分享给他人

---


## 📦 Step 1: 初始化环境

加载框架 + CSS 宽度修复（让图表占满屏幕）

In [1]:
# ========================================
# 移除 JupyterLab 宽度限制（关键！）
# ========================================
from IPython.display import HTML, display

display(HTML("""
<style>
    /* 核心：解除 JupyterLab 4 的宽度限制 */
    .jp-Notebook { 
        --jp-notebook-max-width: 100% !important; 
    }
    
    /* 确保所有输出容器占满宽度 */
    .jp-Notebook-cell, 
    .jp-Cell-outputWrapper, 
    .jp-OutputArea-output, 
    .jp-OutputArea-child { 
        max-width: none !important; 
        width: 100% !important; 
    }
    
    /* Panel 根容器强制铺满 */
    .bk-root, .bk-root > .bk { 
        width: 100% !important; 
        max-width: none !important; 
    }
</style>
"""))

print("✅ JupyterLab 宽度限制已移除")

# ========================================
# 加载框架
# ========================================
import polars as pl
import plotly.express as px
import panel as pn

from src.session import DataSession
from src.dashboard import PanelDashboardBuilder
from src.utils import print_markdown_table

# 初始化 Panel（全局设置 stretch_width）
pn.extension('plotly', sizing_mode='stretch_width')

print("✅ 环境初始化完成")
print("📚 框架版本: Phase 2.0 - Panel Integration")


✅ JupyterLab 宽度限制已移除


✅ 环境初始化完成
📚 框架版本: Phase 2.0 - Panel Integration


## 📂 Step 2: 加载数据

从 Parquet 文件加载数据

In [2]:
# 创建数据会话
session = DataSession()

# 加载数据（替换为你的数据文件）
session.load("insurance_data_cleaned", alias="df")  # 或 session.load("your_data.parquet", alias="df")

# 验证数据
print(f"✅ 数据已加载: {df_df.height:,} 行 × {df_df.width} 列")
print(f"📊 数据预览:\n")
print_markdown_table(df_df.head(5))


✅ 已加载: df_df (insurance_data_cleaned)
✅ 数据已加载: 776,739 行 × 69 列
📊 数据预览:



|   业务年度 |   机构代码 | 机构名称                         |   业务险种 |                 保单号 |   危险单位号 | 支付币别   | 业务来源   | 是否联共保   |   联共保我司占比 |         地址代码 | 标的地址或船名                                                   | 被保险人名称               | 保险起期   | 保险止期   |   风险代码 | 风险代码描述                                         |   风险等级 |      总保额 |   总保费 |   总保费_原币 |   手续费 |   总已决赔款 |   总未决赔款 |    自留保额 |   自留保费 |   自留已决 |   自留未决 |   附加自留保额 |   附加自留保费 |   附加自留已决 |   附加自留未决 |   协议分保额 |   协议分保费 |   协议摊回费用 |   协议已决 |   协议未决 |   成数分保额 |   成数分保费 |   成数摊回费用 |   成数已决 |   成数未决 |   溢额分保额 |   溢额分保费 |   溢额摊回费用 |   溢额已决 |   溢额未决 |   OPENCOVER分保额 |   OPENCOVER分保费 |   OPENCOVER摊回费用 |   OPENCOVER已决 |   OPENCOVER未决 |   临分分保额 |   临分分保费 |   临分摊回费用 |   临分已决 |   临分未决 | 占用性质名称   | 行业类别名称   | 境内境外   |   分入手续费率 |   分入手续费 |   分入出单费比例 |   分入出单费 |   分入经纪费率 |   分入经纪费 | 分入经纪人名称   | 直保代理人名称                           | 归属机构   |
|-----------:|-----------:|:---------------------------------|-----------:|-----------------------:|-------------:|:-----------|:-----------|:-------------|-----------------:|-----------------:|:-----------------------------------------------------------------|:---------------------------|:-----------|:-----------|-----------:|:-----------------------------------------------------|-----------:|------------:|---------:|--------------:|---------:|-------------:|-------------:|------------:|-----------:|-----------:|-----------:|---------------:|---------------:|---------------:|---------------:|-------------:|-------------:|---------------:|-----------:|-----------:|-------------:|-------------:|---------------:|-----------:|-----------:|-------------:|-------------:|---------------:|-----------:|-----------:|------------------:|------------------:|--------------------:|----------------:|----------------:|-------------:|-------------:|---------------:|-----------:|-----------:|:---------------|:---------------|:-----------|---------------:|-------------:|-----------------:|-------------:|---------------:|-------------:|:-----------------|:-----------------------------------------|:-----------|
|       2024 |   44030033 | 深圳分公司工商保险业务部         |       0102 | 3010220244403000000577 |            1 | CNY        | 直接业务   | 非联共保     |            100   | 4403003300008323 | 广东省-深圳市-福田区滨河大道与益田路交界东南皇庭世纪花园1号楼14A | 崔鹏                       | 2024-04-23 | 2027-07-22 |      24107 | 中行渠道财产险类一般行业                             |          1 | 7.8329e+06  |  1921.27 |       1921.27 |     0    |            0 |            0 | 7.8329e+06  |    1921.27 |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 综合商业类     |                | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  |                                          | 深圳       |
|       2016 |   37070004 | 山东分公司潍坊中心支公司银保团队 |       0101 | 3010120163707000000006 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3707000300000714 | 山东省潍坊市滨海经济开发区新海大街99号                           | 潍坊友容实业有限公司       | 2016-01-06 | 2017-01-05 |      11160 | 中行渠道非共保或主共保财产险类一般行业               |          2 | 3.97982e+07 | 27858.8  |      27858.8  |  4178.81 |            0 |            0 | 3.97982e+07 |   27858.8  |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 商业           | 建筑安装业     | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行股份有限公司潍坊分行             | 山东       |
|       2023 |   31030001 | 上海自贸试验区分公司经理室       |       0104 | 3010420233103000001350 |            1 | CNY        | 经纪业务   | 从共         |             10.5 | 3100001700042357 | 新疆维吾尔自治区哈密地区哈密市伊州区                             | 中国广核集团               | 2023-02-01 | 2023-09-30 |      23056 | 市场渠道共保财产险类电厂及水电站（不含联合循环电厂） |          2 | 1.33116e+08 | 21421.9  |      21421.9  |  3406.08 |            0 |            0 | 1.2e+08     |   19311.2  |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  1.31159e+07 |      2110.69 |         675.42 |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 二级工业       | 其他电力生产   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中广核保险经纪有限责任公司               | 上海       |
|       2020 |   33070007 | 浙江分公司金华中心支公司银保团队 |       0101 | 3010120203307000000114 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3307000800003824 | 浙江省金华市兰溪市兰江街道毕家村                                 | 兰溪市三江仓储发展有限公司 | 2020-05-23 | 2021-05-22 |      20107 | 中行渠道财产险类一般行业                             |          3 | 1.23e+06    |  1160.38 |       1160.38 |     0    |            0 |            0 | 1.23e+06    |    1160.38 |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 工业           | 内河货物运输   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行股份有限公司兰溪支行             | 浙江       |
|       2011 |   32010004 | 江苏分公司南京营业部银保团队     |       0102 |  301022010320100000012 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3200001000000077 | 江苏省南京市太平北路106号第四至七层                              | 宏图三胞高科技术有限公司   | 2011-01-01 | 2011-12-31 |      10383 | 中行渠道非共保或主共保财产险类一般行业               |          2 | 2.95e+07    | 14750    |      14750    |  2360    |            0 |            0 | 2.832e+07   |   14160    |          0 |          0 |              0 |              0 |              0 |              0 |     1.18e+06 |          590 |          241.9 |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 商业           | 其他科技服务   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行南京地区鼓楼支行公司板块公司业务 | 江苏       |

## 🤖 Step 3: 生成基本 AI Context

生成数据概览

In [3]:
# 生成基本 AI Context
print("📋 数据结构信息:\n")
print("=" * 80)
print(session.get_ai_context())
print("=" * 80)

print("\n💡 提示: 下一步会生成完整的 AI Prompt")


📋 数据结构信息:

# 📊 当前数据会话

已加载的数据集：

## `df_df` (insurance_data_cleaned)
**数据量：** 776,739 行 × 69 列

**字段：**
- `业务年度` (String)
- `机构代码` (String)
- `机构名称` (String)
- `业务险种` (String)
- `保单号` (String)
- `危险单位号` (String)
- `支付币别` (String)
- `业务来源` (String)
- `是否联共保` (String)
- `联共保我司占比` (Float64)
- `地址代码` (String)
- `标的地址或船名` (String)
- `被保险人名称` (String)
- `保险起期` (String)
- `保险止期` (String)
- `风险代码` (String)
- `风险代码描述` (String)
- `风险等级` (String)
- `总保额` (Float64)
- `总保费` (Float64)
- `总保费_原币` (Float64)
- `手续费` (Float64)
- `总已决赔款` (Float64)
- `总未决赔款` (Float64)
- `自留保额` (Float64)
- `自留保费` (Float64)
- `自留已决` (Float64)
- `自留未决` (Float64)
- `附加自留保额` (Float64)
- `附加自留保费` (Float64)
- `附加自留已决` (Float64)
- `附加自留未决` (Float64)
- `协议分保额` (Float64)
- `协议分保费` (Float64)
- `协议摊回费用` (Float64)
- `协议已决` (Float64)
- `协议未决` (Float64)
- `成数分保额` (Int64)
- `成数分保费` (Int64)
- `成数摊回费用` (Int64)
- `成数已决` (Int64)
- `成数未决` (Int64)
- `溢额分保额` (Float64)
- `溢额分保费` (Float64)
- `溢额摊回费用` (Float64)
- `溢额已决` (Float64)
- `溢额未决` (Float64

## 📋 Step 4: 生成完整 AI Prompt

**运行这个 cell，然后复制全部输出给 AI**

这个 cell 会输出：
- 数据结构信息
- 完整的 Panel Dashboard 使用指南
- 代码模板
- 常见错误对比
- 检查清单


In [4]:
# ========================================
# 📋 给 AI 的完整信息（复制全部输出给 AI）
# ========================================

# 从模板文件读取并执行
import os
template_path = '/Users/harold/working/Jupyter_AI_DataAnalyze/notebooks/templates/step4_standalone.py'

if os.path.exists(template_path):
    with open(template_path, 'r', encoding='utf-8') as f:
        exec(f.read())
else:
    print("⚠️  模板文件未找到，使用简化版本")
    print("\n📋 给 AI 的信息:\n")
    print("=" * 80)
    print("\n## 📊 数据结构\n")
    print(session.get_ai_context())
    print("\n" + "=" * 80)
    print("\n## 📚 必读文档\n")
    print("1. docs/ai_context/main.md")
    print("2. docs/ai_context/PANEL_GUIDE.md")
    print("3. docs/ai_context/AI_QUICK_REFERENCE.md")
    print("\n## 🚨 关键提醒\n")
    print("- ✅ 必须使用 PanelDashboardBuilder")
    print("- ❌ 禁止使用 DashboardBuilder（已废弃）")
    print("- ✅ 必须添加 CSS 宽度修复")
    print("- ✅ 使用 @pn.depends 装饰器")
    print("\n=" * 80)
    print("\n💡 复制上面的内容给 AI，然后说明你的分析需求")


📋 **复制以下所有内容给 AI**

## 📊 数据结构

# 📊 当前数据会话

已加载的数据集：

## `df_df` (insurance_data_cleaned)
**数据量：** 776,739 行 × 69 列

**字段：**
- `业务年度` (String)
- `机构代码` (String)
- `机构名称` (String)
- `业务险种` (String)
- `保单号` (String)
- `危险单位号` (String)
- `支付币别` (String)
- `业务来源` (String)
- `是否联共保` (String)
- `联共保我司占比` (Float64)
- `地址代码` (String)
- `标的地址或船名` (String)
- `被保险人名称` (String)
- `保险起期` (String)
- `保险止期` (String)
- `风险代码` (String)
- `风险代码描述` (String)
- `风险等级` (String)
- `总保额` (Float64)
- `总保费` (Float64)
- `总保费_原币` (Float64)
- `手续费` (Float64)
- `总已决赔款` (Float64)
- `总未决赔款` (Float64)
- `自留保额` (Float64)
- `自留保费` (Float64)
- `自留已决` (Float64)
- `自留未决` (Float64)
- `附加自留保额` (Float64)
- `附加自留保费` (Float64)
- `附加自留已决` (Float64)
- `附加自留未决` (Float64)
- `协议分保额` (Float64)
- `协议分保费` (Float64)
- `协议摊回费用` (Float64)
- `协议已决` (Float64)
- `协议未决` (Float64)
- `成数分保额` (Int64)
- `成数分保费` (Int64)
- `成数摊回费用` (Int64)
- `成数已决` (Int64)
- `成数未决` (Int64)
- `溢额分保额` (Float64)
- `溢额分保费` (Float64)
- `溢额摊回费用` (Float64)
- `溢额已决` (Float6

## 🎯 Step 5: 选择分析维度

选择你想要分析的维度字段

In [5]:
# 👤 选择你要分析的维度
# 修改下面的列表，添加或删除维度

selected_dimensions = [
    '业务年度',
    '业务险种',
    '归属机构'  # 取消注释以启用
    # '境内境外',  # 取消注释以启用
]

print(f"✅ 已选择 {len(selected_dimensions)} 个维度:")
for dim in selected_dimensions:
    if dim in df_df.columns:
        n_unique = df_df[dim].n_unique()
        print(f"  - {dim} ({n_unique:,} 个唯一值)")
    else:
        print(f"  ⚠️  {dim} - 不存在于数据中")

print("\n💡 下一步: 创建 Panel 仪表盘")


✅ 已选择 3 个维度:
  - 业务年度 (25 个唯一值)
  - 业务险种 (26 个唯一值)
  - 归属机构 (30 个唯一值)

💡 下一步: 创建 Panel 仪表盘


## 📊 Step 6: 创建 Panel 仪表盘

自动创建仪表盘控件，然后让 AI 生成分析逻辑

In [6]:
# 创建 Panel 仪表盘（自动生成控件）
dashboard = PanelDashboardBuilder.from_data(
    df_df,
    dimensions=selected_dimensions,
    title="📊 数据分析仪表盘"
)

print("\n💡 下一步: 让 AI 生成分析逻辑")
print("\n📝 给 AI 的提示:")
print("="  * 80)
print("请使用 Panel Dashboard 生成分析代码。")
print(f"已选择维度: {selected_dimensions}")
print("需求: [描述你的分析需求，例如: 各险种保费排名 Top 10]")
print("="  * 80)


🎨 从数据创建仪表盘: 📊 数据分析仪表盘
📊 数据维度: 776,739 行 × 69 列
🔧 配置维度字段: 业务年度, 业务险种, 归属机构



  ✅ 业务年度: MultiChoice (25 个选项 + 全选, 默认选 1 个)
  ✅ 业务险种: MultiChoice (26 个选项 + 全选, 默认选 1 个)
  ✅ 归属机构: MultiChoice (30 个选项 + 全选, 默认选 1 个)

  ✅ 聚合维度选择器: Select (3 个维度可选, 默认: 业务年度)

✅ 仪表盘控件创建完成 (4 个控件)
💡 下一步: 使用 dashboard.set_update_function(your_function)

📄 等效代码（可复制用于自定义）:

```python
import panel as pn
import polars as pl

dashboard = PanelDashboardBuilder(title="📊 数据分析仪表盘")

# 示例: 业务年度
widget = pn.widgets.Select(name='业务年度', options=[...])
dashboard.widgets['业务年度'] = widget

# 示例: 业务险种
widget = pn.widgets.Select(name='业务险种', options=[...])
dashboard.widgets['业务险种'] = widget

# 定义更新函数
@pn.depends(*dashboard.widgets.values())
def update(*args):
    # 你的分析逻辑
    return fig

dashboard.set_update_function(update)
dashboard.show()  # Jupyter 显示
dashboard.save('output.html')  # 导出 HTML
```



💡 下一步: 让 AI 生成分析逻辑

📝 给 AI 的提示:
请使用 Panel Dashboard 生成分析代码。
已选择维度: ['业务年度', '业务险种', '归属机构']
需求: [描述你的分析需求，例如: 各险种保费排名 Top 10]


## 🤖 Step 7: AI 生成分析逻辑

**在这里粘贴 AI 生成的代码**

AI 应该生成：
1. `@pn.depends` 装饰的更新函数
2. 数据过滤和聚合逻辑
3. Plotly 图表
4. `dashboard.show()` 和 `dashboard.save()`


## 📤 Step 8: 导出和分享

如果 AI 代码中没有包含 `dashboard.save()`，手动导出：

In [7]:
import panel as pn
import polars as pl
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from src.utils import print_markdown_table

# ==============================================================================
# 最终版：多级分组 + 正负值相对堆叠 (barmode='relative')
# ==============================================================================
@pn.depends(*dashboard.widgets.values())
def update_dashboard(*args):
    # --- 1. 获取控件 ---
    values = {name: w.value for name, w in dashboard.widgets.items()}
    group_col = values.get('_aggregation_dimension', '业务险种')
    
    # --- 2. 动态过滤 ---
    filters = []
    for dim, val in values.items():
        if dim == '_aggregation_dimension': continue
        if isinstance(val, list):
            if '全选' not in val: filters.append(pl.col(dim).is_in(val))
        elif val != '全选':
            filters.append(pl.col(dim) == val)
            
    filtered = df_df
    for f in filters: filtered = filtered.filter(f)
    
    if filtered.height == 0: return f"⚠️ 按【{group_col}】筛选无数据"

    # --- 3. 聚合计算 ---
    agg_exprs = [
        # 保费
        pl.col('总保费').sum(),
        pl.col('自留保费').sum(),
        # 赔款
        (pl.col('总已决赔款') + pl.col('总未决赔款')).sum().alias('毛赔款'),
        (pl.col('自留已决') + pl.col('自留未决')).sum().alias('自留赔款'),
        # 费用
        pl.col('手续费').sum().alias('直保手续费'),
        # 分入费用
        (pl.col('分入手续费').fill_null(0) + pl.col('分入出单费').fill_null(0) + pl.col('分入经纪费').fill_null(0)).sum().alias('总分入费用'),
        # 摊回费用
        (pl.col('协议摊回费用').fill_null(0) + pl.col('成数摊回费用').fill_null(0) + pl.col('溢额摊回费用').fill_null(0) + 
         pl.col('OPENCOVER摊回费用').fill_null(0) + pl.col('临分摊回费用').fill_null(0)).sum().alias('总摊回费用')
    ]
    
    result = filtered.group_by(group_col).agg(agg_exprs)
    
    # --- 4. 衍生指标 ---
    df_plot = result.to_pandas()
    div = 10000 # 单位：万元
    
    # [1] 保费
    df_plot['毛保费'] = df_plot['总保费'] / div
    df_plot['自留保费'] = df_plot['自留保费'] / div
    df_plot['分出保费'] = df_plot['毛保费'] - df_plot['自留保费']
    
    # [2] 成本 (含分入费用)
    df_plot['总获取成本'] = df_plot['直保手续费'] + df_plot['总分入费用']
    df_plot['毛成本'] = (df_plot['毛赔款'] + df_plot['总获取成本']) / div
    df_plot['自留成本'] = (df_plot['自留赔款'] + (df_plot['总获取成本'] - df_plot['总摊回费用'])) / div
    df_plot['分出成本'] = df_plot['毛成本'] - df_plot['自留成本']
    
    # [3] 利润 (可能为负)
    df_plot['毛利润'] = df_plot['毛保费'] - df_plot['毛成本']
    df_plot['自留利润'] = df_plot['自留保费'] - df_plot['自留成本']
    df_plot['分出利润'] = df_plot['分出保费'] - df_plot['分出成本']
    
    # [4] 比率
    df_plot['分出比例'] = (df_plot['分出保费'] / df_plot['毛保费'] * 100).fillna(0)
    df_plot['毛成本率'] = (df_plot['毛成本'] / df_plot['毛保费'] * 100).fillna(0)
    df_plot['自留成本率'] = (df_plot['自留成本'] / df_plot['自留保费'] * 100).fillna(0)
    df_plot['分出成本率'] = (df_plot['分出成本'] / df_plot['分出保费'] * 100).fillna(0)

    # 排序：按名称排序
    df_plot = df_plot.sort_values(group_col, ascending=True)
    
    # --- 5. 构建多级 X 轴 ---
    # 分组标签
    x_prem = [df_plot[group_col].tolist(), ['保费'] * len(df_plot)]
    x_prof = [df_plot[group_col].tolist(), ['利润'] * len(df_plot)]
    
    # --- 6. 绘图 ---
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # === 栈 1: 保费组 (通常全为正) ===
    # 自留保费
    fig.add_trace(go.Bar(
        x=x_prem, y=df_plot['自留保费'],
        name='自留保费',
        marker_color='#1f77b4', # 深蓝
        hovertemplate='<b>%{x[0]}</b><br>自留保费: %{y:,.0f}万',
    ), secondary_y=False)
    
    # 分出保费
    fig.add_trace(go.Bar(
        x=x_prem, y=df_plot['分出保费'],
        name='分出保费',
        marker_color='#aec7e8', # 浅蓝
        hovertemplate='<b>%{x[0]}</b><br>分出保费: %{y:,.0f}万',
    ), secondary_y=False)

    # === 栈 2: 利润组 (可能有正有负) ===
    # 自留利润 (正负皆可)
    fig.add_trace(go.Bar(
        x=x_prof, y=df_plot['自留利润'],
        name='自留利润',
        marker_color='#ff7f0e', # 深橙
        hovertemplate='<b>%{x[0]}</b><br>自留利润: %{y:,.0f}万',
    ), secondary_y=False)
    
    # 分出利润 (正负皆可)
    fig.add_trace(go.Bar(
        x=x_prof, y=df_plot['分出利润'],
        name='分出利润',
        marker_color='#ffbb78', # 浅橙
        hovertemplate='<b>%{x[0]}</b><br>分出利润: %{y:,.0f}万',
    ), secondary_y=False)

    # === 折线图：比率 (对齐到“保费”列) ===
    fig.add_trace(go.Scatter(
        x=x_prem, y=df_plot['自留成本率'],
        name='自留成本率(%)', mode='lines+markers',
        line=dict(color='#d62728', width=2)
    ), secondary_y=True)
    
    fig.add_trace(go.Scatter(
        x=x_prem, y=df_plot['分出比例'],
        name='分出比例(%)', mode='lines',
        line=dict(color='black', dash='dot', width=1)
    ), secondary_y=True)
    
    fig.add_trace(go.Scatter(
        x=x_prem, y=df_plot['毛成本率'],
        name='毛成本率(%)', mode='lines',
        visible='legendonly',
        line=dict(color='green', width=1)
    ), secondary_y=True)

    # --- 7. 布局优化 (关键设置) ---
    fig.update_layout(
        title=f'<b>{group_col} 深度经营分析</b> (多级分组+正负堆叠)',
        height=700, 
        autosize=True,
        # ✅ 关键点：使用 'relative' 模式
        # 这确保了正数堆在0之上，负数堆在0之下，互不抵消，都从0开始
        barmode='relative', 
        hovermode="closest",
        legend=dict(orientation="h", y=1.02, x=0.5, xanchor="center"),
        margin=dict(t=80, l=20, r=20, b=80),
        xaxis=dict(title_text="")
    )
    
    # 添加 0 轴参考线 (黑色，显眼)
    fig.add_hline(y=0, line_width=1, line_color="black")
    
    fig.update_yaxes(title_text="金额 (万元)", secondary_y=False)
    fig.update_yaxes(title_text="比率 (%)", secondary_y=True) # 自适应缩放
    
    # --- 8. 表格 ---
    print(f"## 📊 {group_col} 全指标明细 (按名称排序)")
    cols = ['毛保费', '分出保费', '分出比例', 
            '毛利润', '自留利润', '分出利润',
            '自留成本率']
    display_df = df_plot[[group_col] + cols].round(1)
    print_markdown_table(pl.from_pandas(display_df))

    return fig

# 绑定并显示
dashboard.set_update_function(update_dashboard)
dashboard.show()

## 📊 业务年度 全指标明细 (按名称排序)


|   业务年度 |   毛保费 |   分出保费 |   分出比例 |   毛利润 |   自留利润 |   分出利润 |   自留成本率 |
|-----------:|---------:|-----------:|-----------:|---------:|-----------:|-----------:|-------------:|
|       2001 |      8.7 |        8.3 |       96   |      8.7 |        2.8 |        5.9 |       -715.8 |
|       2002 |   1185.4 |      809.9 |       68.3 |   1007   |      480.6 |      526.4 |        -28   |
|       2003 |   3368.8 |     1919.4 |       57   |   2222.9 |     1324   |      898.9 |          8.6 |
|       2004 |   4878.6 |     2692.5 |       55.2 |   3981.1 |     2140.3 |     1840.8 |          2.1 |
|       2005 |   4969.1 |     2316.8 |       46.6 |   3749.6 |     2635.4 |     1114.2 |          0.6 |
|       2006 |   9674.6 |     5779.3 |       59.7 |  -3060.2 |     2207.5 |    -5267.8 |         43.3 |
|       2007 |  14023.8 |     7303   |       52.1 |   2307.1 |     3922.7 |    -1615.6 |         41.6 |
|       2008 |  26765   |    12931.6 |       48.3 |  11619.5 |     8121.8 |     3497.7 |         41.3 |
|       2009 |  38149.3 |    18240.4 |       47.8 |  19895.4 |    12657.2 |     7238.2 |         36.4 |
|       2010 |  52258.3 |    25428   |       48.7 |  11610.8 |    11346.3 |      264.5 |         57.7 |
|       2011 |  53245.7 |    20325.7 |       38.2 |  27923.4 |    24002   |     3921.3 |         27.1 |
|       2012 |  62265.4 |    22009.2 |       35.3 |  29714.9 |    26308   |     3406.8 |         34.6 |
|       2013 |  66866.2 |    23284.9 |       34.8 |  14731   |    19658.4 |    -4927.5 |         54.9 |
|       2014 |  76865.6 |    28937.6 |       37.6 |  42131.2 |    33381.7 |     8749.5 |         30.4 |
|       2015 |  69213.2 |    24777.8 |       35.8 |  23977.1 |    24559.8 |     -582.7 |         44.7 |
|       2016 |  90757.2 |    41938.8 |       46.2 |  48377.3 |    32346.9 |    16030.4 |         33.7 |
|       2017 |  77821.6 |    33738.7 |       43.4 |  19721.4 |    17714.6 |     2006.8 |         59.8 |
|       2018 |  87299.8 |    35447.3 |       40.6 |  31529.3 |    26439.1 |     5090.2 |         49   |
|       2019 |  91188.4 |    36101.2 |       39.6 |  36214.4 |    29756.1 |     6458.3 |         46   |
|       2020 |  85256.8 |    35201.5 |       41.3 |  28651.1 |    21325.4 |     7325.7 |         57.4 |
|       2021 |  94894.3 |    35804.6 |       37.7 |   7836.5 |    29022.6 |   -21186.1 |         50.9 |
|       2022 |  94553   |    33124.1 |       35   |  45694   |    38656.1 |     7037.9 |         37.1 |
|       2023 |  97699.9 |    36036.3 |       36.9 |  41872.6 |    32378.6 |     9494   |         47.5 |
|       2024 |  99925.8 |    36944.9 |       37   |  47895.7 |    36777.6 |    11118.1 |         41.6 |
|       2025 |  80041.4 |    29129.5 |       36.4 |  46405.6 |    35591.3 |    10814.2 |         30.1 |

## 📊 业务险种 全指标明细 (按名称排序) 
## 📊 业务险种 全指标明细 (按名称排序) 
## 📊 业务险种 全指标明细 (按名称排序) 
## 📊 业务险种 全指标明细 (按名称排序)

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] FlexBox(objects=[MultiChoice(name='📊 业务年度'...], sizing_mode='stretch_width')
    [2] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')

In [8]:
layout = dashboard.build_layout()
# 直接保存 layout
layout.save("分析2.html", embed=True)

## 📊 业务年度 全指标明细 (按名称排序)


|   业务年度 |   毛保费 |   分出保费 |   分出比例 |   毛利润 |   自留利润 |   分出利润 |   自留成本率 |
|-----------:|---------:|-----------:|-----------:|---------:|-----------:|-----------:|-------------:|
|       2001 |      8.7 |        8.3 |       96   |      8.7 |        2.8 |        5.9 |       -715.8 |
|       2002 |   1185.4 |      809.9 |       68.3 |   1007   |      480.6 |      526.4 |        -28   |
|       2003 |   3368.8 |     1919.4 |       57   |   2222.9 |     1324   |      898.9 |          8.6 |
|       2004 |   4878.6 |     2692.5 |       55.2 |   3981.1 |     2140.3 |     1840.8 |          2.1 |
|       2005 |   4969.1 |     2316.8 |       46.6 |   3749.6 |     2635.4 |     1114.2 |          0.6 |
|       2006 |   9674.6 |     5779.3 |       59.7 |  -3060.2 |     2207.5 |    -5267.8 |         43.3 |
|       2007 |  14023.8 |     7303   |       52.1 |   2307.1 |     3922.7 |    -1615.6 |         41.6 |
|       2008 |  26765   |    12931.6 |       48.3 |  11619.5 |     8121.8 |     3497.7 |         41.3 |
|       2009 |  38149.3 |    18240.4 |       47.8 |  19895.4 |    12657.2 |     7238.2 |         36.4 |
|       2010 |  52258.3 |    25428   |       48.7 |  11610.8 |    11346.3 |      264.5 |         57.7 |
|       2011 |  53245.7 |    20325.7 |       38.2 |  27923.4 |    24002   |     3921.3 |         27.1 |
|       2012 |  62265.4 |    22009.2 |       35.3 |  29714.9 |    26308   |     3406.8 |         34.6 |
|       2013 |  66866.2 |    23284.9 |       34.8 |  14731   |    19658.4 |    -4927.5 |         54.9 |
|       2014 |  76865.6 |    28937.6 |       37.6 |  42131.2 |    33381.7 |     8749.5 |         30.4 |
|       2015 |  69213.2 |    24777.8 |       35.8 |  23977.1 |    24559.8 |     -582.7 |         44.7 |
|       2016 |  90757.2 |    41938.8 |       46.2 |  48377.3 |    32346.9 |    16030.4 |         33.7 |
|       2017 |  77821.6 |    33738.7 |       43.4 |  19721.4 |    17714.6 |     2006.8 |         59.8 |
|       2018 |  87299.8 |    35447.3 |       40.6 |  31529.3 |    26439.1 |     5090.2 |         49   |
|       2019 |  91188.4 |    36101.2 |       39.6 |  36214.4 |    29756.1 |     6458.3 |         46   |
|       2020 |  85256.8 |    35201.5 |       41.3 |  28651.1 |    21325.4 |     7325.7 |         57.4 |
|       2021 |  94894.3 |    35804.6 |       37.7 |   7836.5 |    29022.6 |   -21186.1 |         50.9 |
|       2022 |  94553   |    33124.1 |       35   |  45694   |    38656.1 |     7037.9 |         37.1 |
|       2023 |  97699.9 |    36036.3 |       36.9 |  41872.6 |    32378.6 |     9494   |         47.5 |
|       2024 |  99925.8 |    36944.9 |       37   |  47895.7 |    36777.6 |    11118.1 |         41.6 |
|       2025 |  80041.4 |    29129.5 |       36.4 |  46405.6 |    35591.3 |    10814.2 |         30.1 |

  0%|                                                                           | 0/3 [00:00<?, ?it/s]## 📊 归属机构 全指标明细 (按名称排序)


| 归属机构   |   毛保费 |   分出保费 |   分出比例 |   毛利润 |   自留利润 |   分出利润 |   自留成本率 |
|:-----------|---------:|-----------:|-----------:|---------:|-----------:|-----------:|-------------:|
| 上海       |  68089.6 |    31087.4 |       45.7 |  -8986.6 |    13897.5 |   -22884.1 |         62.4 |
| 云南       |  27299.5 |    13522.4 |       49.5 |  10968.2 |     7454.3 |     3513.9 |         45.9 |
| 其他       |   1210.5 |      190.9 |       15.8 |    851.8 |      776.2 |       75.7 |         23.9 |
| 内蒙古     |  37150.9 |    15427.9 |       41.5 |  14609   |    11548.6 |     3060.4 |         46.8 |
| 北京       |  70472.1 |    33316.2 |       47.3 |  41699   |    27686.1 |    14012.9 |         25.5 |
| 厦门       |   4962.4 |     1588.4 |       32   |   1390.1 |     1274.2 |      115.9 |         62.2 |
| 四川       |  45669.7 |    24073   |       52.7 |  17304.4 |    11423.1 |     5881.4 |         47.1 |
| 大连       |  19692.2 |     7999.5 |       40.6 |  12852.4 |     9203.5 |     3648.9 |         21.3 |
| 天津       |  32476.5 |    17082.6 |       52.6 |   1153.5 |     7594.1 |    -6440.5 |         50.7 |
| 宁波       |  26089.1 |     8539.8 |       32.7 |   7270.7 |     7038.5 |      232.2 |         59.9 |
| 安徽       |  36574.5 |    11045.8 |       30.2 |  16501   |    15008.7 |     1492.3 |         41.2 |
| 山东       |  76473.9 |    27127.8 |       35.5 |  46140.8 |    36651.4 |     9489.4 |         25.7 |
| 山西       |   6072.7 |     2148.3 |       35.4 |   1754.7 |     2703.4 |     -948.7 |         31.1 |
| 广东       | 167723   |    62638.8 |       37.3 |  56896.9 |    48746.2 |     8150.7 |         53.6 |
| 广西       |  17013.6 |     5322   |       31.3 |   3835.5 |     4403.1 |     -567.6 |         62.3 |
| 总公司     | 143490   |    89241.1 |       62.2 |  57378.5 |    27062.1 |    30316.4 |         50.1 |
| 新疆       |  10180.5 |     4073.3 |       40   |   5649.4 |     4148.1 |     1501.3 |         32.1 |
| 江苏       | 122898   |    39583.1 |       32.2 |  71040.4 |    55964.4 |    15076   |         32.8 |
| 江西       |  21950.9 |     7528.8 |       34.3 |   8607.6 |     7855.1 |      752.5 |         45.5 |
| 河北       |  33975.5 |    14844.5 |       43.7 |  14163.2 |    11493.6 |     2669.5 |         39.9 |
| 河南       |  43944.4 |    12767   |       29.1 |  18775.4 |    15514.1 |     3261.4 |         50.2 |
| 浙江       | 108564   |    38704.8 |       35.7 |  36638.5 |    37014.4 |     -375.9 |         47   |
| 深圳       |  47909.6 |    15536.6 |       32.4 |   8927.2 |    18260.9 |    -9333.6 |         43.6 |
| 湖北       |  29398.2 |    12790.8 |       43.5 |  15842.3 |    11567.8 |     4274.5 |         30.3 |
| 湖南       |  31368.7 |     5699   |       18.2 |  14764.2 |    13885.7 |      878.5 |         45.9 |
| 福建       |  35612   |    10337.7 |       29   |  14336.3 |    12336.6 |     1999.6 |         51.2 |
| 苏州       |  38383.8 |    10213.2 |       26.6 |  26209.1 |    22050   |     4159.2 |         21.7 |
| 辽宁       |  34848.1 |    10980.6 |       31.5 |  18211.7 |    14842   |     3369.7 |         37.8 |
| 陕西       |  31603.1 |    10725   |       33.9 |  17248.3 |    13730.8 |     3517.4 |         34.2 |
| 黑龙江     |  12078.2 |     6095   |       50.5 |  -6016.6 |     1622.5 |    -7639.1 |         72.9 |

## 📊 归属机构 全指标明细 (按名称排序)


...

|   业务险种 |   毛保费 |   分出保费 |   分出比例 |   毛利润 |   自留利润 |   分出利润 |   自留成本率 |
|-----------:|---------:|-----------:|-----------:|---------:|-----------:|-----------:|-------------:|
|       0101 | 135662   |    21863.9 |       16.1 |  86913.9 |    76943.6 |     9970.3 |         32.4 |
|       0102 | 370163   |    93778   |       25.3 | 163871   |   145149   |    18721.6 |         47.5 |
|       0103 |   4393.9 |     1578.7 |       35.9 |   3308.1 |     2434.7 |      873.4 |         13.5 |
|       0104 | 430887   |   216109   |       50.2 |  97800.6 |   109875   |   -12074.3 |         48.8 |
|       0105 | 106698   |    50432.5 |       47.3 |  34324.4 |    28645.8 |     5678.5 |         49.1 |
|       0108 |  17151.3 |     7462.8 |       43.5 |  16652.9 |    11794.2 |     4858.8 |        -21.7 |
|       0110 |   4587.2 |      578.1 |       12.6 |   3682.1 |     3311.3 |      370.8 |         17.4 |
|       0111 |   6048   |     1028.1 |       17   |   3520.7 |     3167.5 |      353.2 |         36.9 |
|       0112 |  11564.9 |     4062.9 |       35.1 |   7669.6 |     5911.3 |     1758.3 |         21.2 |
|       0113 |     45.9 |       22.5 |       49.1 |     39   |       24.6 |       14.4 |         -5.3 |
|       0114 |    918.9 |      465   |       50.6 |    571.1 |      316.1 |      255   |         30.4 |
|       0115 |   3105.5 |     1965.6 |       63.3 |   -699.1 |    -1599   |      899.9 |        240.3 |
|       0116 |   6504.9 |     4375.8 |       67.3 |   2034.9 |      772.4 |     1262.5 |         63.7 |
|       0117 |  18887.1 |     1894.9 |       10   |  12967.7 |    11936   |     1031.7 |         29.8 |
|       0119 |    250.4 |       39.6 |       15.8 |    191.3 |      170.9 |       20.4 |         18.9 |
|       0120 |   9873.6 |       48.8 |        0.5 |   2936.5 |     2903.8 |       32.7 |         70.4 |
|       0121 |  12026.4 |        0   |        0   |   7863.6 |     7863.6 |        0   |         34.6 |
|       0122 |   2573.7 |     1166.5 |       45.3 |   1688.8 |     1074.2 |      614.5 |         23.7 |
|       0123 |    568.1 |        3.2 |        0.6 |    382.8 |      379.6 |        3.2 |         32.8 |
|       0124 |  16047.9 |        0   |        0   |  12635   |    12635   |        0   |         21.3 |
|       0701 | 189987   |   122883   |       64.7 |  67686.6 |    38335.7 |    29350.9 |         42.9 |
|       0702 |  15114.9 |     8709.9 |       57.6 |   9023.4 |     4077.4 |     4946   |         36.3 |
|       0705 |  12478.9 |     8690.4 |       69.6 |   5991.4 |     2620.3 |     3371.2 |         30.8 |
|       0706 |   4295.7 |     2538.2 |       59.1 |   2726.3 |     1745.1 |      981.2 |          0.7 |
|       0707 |      1.7 |        1.3 |       80   |      1.7 |        0.5 |        1.1 |        -56.3 |
|       0911 |   3340.6 |      532.2 |       15.9 |   2233.1 |     2268.2 |      -35.2 |         19.2 |

## 📊 业务年度 全指标明细 (按名称排序)


|   业务年度 |   毛保费 |   分出保费 |   分出比例 |   毛利润 |   自留利润 |   分出利润 |   自留成本率 |
|-----------:|---------:|-----------:|-----------:|---------:|-----------:|-----------:|-------------:|
|       2001 |      8.7 |        8.3 |       96   |      8.7 |        2.8 |        5.9 |       -715.8 |
|       2002 |   1185.4 |      809.9 |       68.3 |   1007   |      480.6 |      526.4 |        -28   |
|       2003 |   3368.8 |     1919.4 |       57   |   2222.9 |     1324   |      898.9 |          8.6 |
|       2004 |   4878.6 |     2692.5 |       55.2 |   3981.1 |     2140.3 |     1840.8 |          2.1 |
|       2005 |   4969.1 |     2316.8 |       46.6 |   3749.6 |     2635.4 |     1114.2 |          0.6 |
|       2006 |   9674.6 |     5779.3 |       59.7 |  -3060.2 |     2207.5 |    -5267.8 |         43.3 |
|       2007 |  14023.8 |     7303   |       52.1 |   2307.1 |     3922.7 |    -1615.6 |         41.6 |
|       2008 |  26765   |    12931.6 |       48.3 |  11619.5 |     8121.8 |     3497.7 |         41.3 |
|       2009 |  38149.3 |    18240.4 |       47.8 |  19895.4 |    12657.2 |     7238.2 |         36.4 |
|       2010 |  52258.3 |    25428   |       48.7 |  11610.8 |    11346.3 |      264.5 |         57.7 |
|       2011 |  53245.7 |    20325.7 |       38.2 |  27923.4 |    24002   |     3921.3 |         27.1 |
|       2012 |  62265.4 |    22009.2 |       35.3 |  29714.9 |    26308   |     3406.8 |         34.6 |
|       2013 |  66866.2 |    23284.9 |       34.8 |  14731   |    19658.4 |    -4927.5 |         54.9 |
|       2014 |  76865.6 |    28937.6 |       37.6 |  42131.2 |    33381.7 |     8749.5 |         30.4 |
|       2015 |  69213.2 |    24777.8 |       35.8 |  23977.1 |    24559.8 |     -582.7 |         44.7 |
|       2016 |  90757.2 |    41938.8 |       46.2 |  48377.3 |    32346.9 |    16030.4 |         33.7 |
|       2017 |  77821.6 |    33738.7 |       43.4 |  19721.4 |    17714.6 |     2006.8 |         59.8 |
|       2018 |  87299.8 |    35447.3 |       40.6 |  31529.3 |    26439.1 |     5090.2 |         49   |
|       2019 |  91188.4 |    36101.2 |       39.6 |  36214.4 |    29756.1 |     6458.3 |         46   |
|       2020 |  85256.8 |    35201.5 |       41.3 |  28651.1 |    21325.4 |     7325.7 |         57.4 |
|       2021 |  94894.3 |    35804.6 |       37.7 |   7836.5 |    29022.6 |   -21186.1 |         50.9 |
|       2022 |  94553   |    33124.1 |       35   |  45694   |    38656.1 |     7037.9 |         37.1 |
|       2023 |  97699.9 |    36036.3 |       36.9 |  41872.6 |    32378.6 |     9494   |         47.5 |
|       2024 |  99925.8 |    36944.9 |       37   |  47895.7 |    36777.6 |    11118.1 |         41.6 |
|       2025 |  80041.4 |    29129.5 |       36.4 |  46405.6 |    35591.3 |    10814.2 |         30.1 |

## 📊 业务年度 全指标明细 (按名称排序)


|   业务年度 |   毛保费 |   分出保费 |   分出比例 |   毛利润 |   自留利润 |   分出利润 |   自留成本率 |
|-----------:|---------:|-----------:|-----------:|---------:|-----------:|-----------:|-------------:|
|       2001 |      8.7 |        8.3 |       96   |      8.7 |        2.8 |        5.9 |       -715.8 |
|       2002 |   1185.4 |      809.9 |       68.3 |   1007   |      480.6 |      526.4 |        -28   |
|       2003 |   3368.8 |     1919.4 |       57   |   2222.9 |     1324   |      898.9 |          8.6 |
|       2004 |   4878.6 |     2692.5 |       55.2 |   3981.1 |     2140.3 |     1840.8 |          2.1 |
|       2005 |   4969.1 |     2316.8 |       46.6 |   3749.6 |     2635.4 |     1114.2 |          0.6 |
|       2006 |   9674.6 |     5779.3 |       59.7 |  -3060.2 |     2207.5 |    -5267.8 |         43.3 |
|       2007 |  14023.8 |     7303   |       52.1 |   2307.1 |     3922.7 |    -1615.6 |         41.6 |
|       2008 |  26765   |    12931.6 |       48.3 |  11619.5 |     8121.8 |     3497.7 |         41.3 |
|       2009 |  38149.3 |    18240.4 |       47.8 |  19895.4 |    12657.2 |     7238.2 |         36.4 |
|       2010 |  52258.3 |    25428   |       48.7 |  11610.8 |    11346.3 |      264.5 |         57.7 |
|       2011 |  53245.7 |    20325.7 |       38.2 |  27923.4 |    24002   |     3921.3 |         27.1 |
|       2012 |  62265.4 |    22009.2 |       35.3 |  29714.9 |    26308   |     3406.8 |         34.6 |
|       2013 |  66866.2 |    23284.9 |       34.8 |  14731   |    19658.4 |    -4927.5 |         54.9 |
|       2014 |  76865.6 |    28937.6 |       37.6 |  42131.2 |    33381.7 |     8749.5 |         30.4 |
|       2015 |  69213.2 |    24777.8 |       35.8 |  23977.1 |    24559.8 |     -582.7 |         44.7 |
|       2016 |  90757.2 |    41938.8 |       46.2 |  48377.3 |    32346.9 |    16030.4 |         33.7 |
|       2017 |  77821.6 |    33738.7 |       43.4 |  19721.4 |    17714.6 |     2006.8 |         59.8 |
|       2018 |  87299.8 |    35447.3 |       40.6 |  31529.3 |    26439.1 |     5090.2 |         49   |
|       2019 |  91188.4 |    36101.2 |       39.6 |  36214.4 |    29756.1 |     6458.3 |         46   |
|       2020 |  85256.8 |    35201.5 |       41.3 |  28651.1 |    21325.4 |     7325.7 |         57.4 |
|       2021 |  94894.3 |    35804.6 |       37.7 |   7836.5 |    29022.6 |   -21186.1 |         50.9 |
|       2022 |  94553   |    33124.1 |       35   |  45694   |    38656.1 |     7037.9 |         37.1 |
|       2023 |  97699.9 |    36036.3 |       36.9 |  41872.6 |    32378.6 |     9494   |         47.5 |
|       2024 |  99925.8 |    36944.9 |       37   |  47895.7 |    36777.6 |    11118.1 |         41.6 |
|       2025 |  80041.4 |    29129.5 |       36.4 |  46405.6 |    35591.3 |    10814.2 |         30.1 |